In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('AcademicBooksDB.sqlite')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON;')
print("Created and/or Opened database successfully")
cursor.close()

Created and/or Opened database successfully


### Task 3 - Business Logic

In a three-tier architecture we often use stored procedures to ensure that DML operations such as INSERT, UPDATE, or DELETE commands execute correctly. These procedures also serve to ensure input have been appropriately validated and that any possible exceptions have been dealt with. Your task is to write the code for selected procedures in the business logic tier. 

**For each procedure the following will apply:**
1. The signature will be provided. This signature may not be changed.
2. Some parameters might be optional. You need to ensure you add appropriate logic to deal with all possible cases correctly
3. All procedures must return appropriate messages to the calling function to communicate the result of running the code (was the call successful? Did an error occur? With what error message?)
4. A test call to each procedure is provided in cells following the procedure. These should work as expected if your code was correct

#### Stored Procedure 1: Adding a New Customer
- To register, a customer **must** provide their Name and Surname. 
- Address information is optional but must be provided before a first order can be shipped
- If an address is provided it must include the values for Adress, City, and Country, a customer cannot provide and address without specifying both City and Country. If this information is missing or incomplete the customer record needs to be created but the calling application should receive a message that the address information was not stored.
- Providing a phone number is **always** optional
- On successful creation of a new customer the autogenerated ID for the new customer must always be returned as part of the success message

In [4]:
q = """
SELECT *
FROM Customers"""
pd.read_sql(q, conn)

,CustID,FirstName,LastName,Address,City,Country,Phone
0,1,Johan,van Niekerk,123 Kirkegata,Kristiansand,Norway,0047 91234432
1,2,Rayne,Reid,123 Hollendergata,Oslo,Norway,0047 12345678
2,3,Tor,Schultz,Smith Strasse,Berlin,Germany,0049 12345678
3,4,Buck,Rodgers,None,None,None,None
4,5,Darth,Vader,None,None,None,None


In [24]:
a = None
b = 0
if b:
    print('what i wanna print')    

In [33]:
def NewCustomer(Name, Surname, Address = None, City = None, Country = None, Phone = None):
    # Your code should go here
    
    if (not Address) or (not City) or (not Country):
        Address = None
        City = None
        Country = None
        print('Warning: Address not updated')
    
    SQL = '''INSERT INTO Customers(FirstName, LastName, Address, City, Country, Phone)
             VALUES(?,?,?,?,?,?)'''
    cursor=conn.cursor()
    cursor.execute(SQL,(Name, Surname, Address, City, Country, Phone,))
    cust_id = cursor.lastrowid
    
    return cust_id    

In [32]:
NewCustomer('Hello')

TypeError: NewCustomer() missing 1 required positional argument: 'Surname'

In [34]:
NewCustomer('Hello', 'Friend', 'aabab', 'ababa', 'askjnfkjdnf')

10

In [28]:
NewCustomer('Hello', 'Friend', Country ='ababa')

In [30]:
NewCustomer('John', 'Smith', Address ='ababa', Phone='8756666666')

In [35]:
q = """
SELECT *
FROM Customers"""
pd.read_sql(q, conn)

,CustID,FirstName,LastName,Address,City,Country,Phone
0,1,Johan,van Niekerk,123 Kirkegata,Kristiansand,Norway,0047 91234432
1,2,Rayne,Reid,123 Hollendergata,Oslo,Norway,0047 12345678
2,3,Tor,Schultz,Smith Strasse,Berlin,Germany,0049 12345678
3,4,Buck,Rodgers,None,None,None,None
4,5,Darth,Vader,None,None,None,None
5,6,Hello,Friend,None,None,None,None
6,7,Hello,Friend,aabab,ababa,askjnfkjdnf,None
7,8,Hello,Friend,None,None,None,None
8,9,John,Smith,None,None,None,8756666666
9,10,Hello,Friend,aabab,ababa,askjnfkjdnf,None


#### Stored Procedure 2: Updating an existing Customer
- To Update a Customer record the customer ID must be provided
- A customer may only change their Address, City, Country, or Phone Number
- Address, City, and Country information must always be updated together
- Phone Number is always optional
- A phone number may only contain numerical values 
- A phone number **might** start with a country code. For example a South African number might start with 0027 and a Norwegian number with 0047. You **do not** have to verify that the correct country code was used

In [40]:
def UpdateCustomer(CustID, Address = None, City = None, Country = None, Phone = None):
    # Your code should go here
    
    if Phone and Phone.isdecimal():
        print('Yes we updated phone number')
    else:
        print('Phone does not contain only numerical values')
        Phone = None        
    
    if (not Address) or (not City) or (not Country):
        Address = None
        City = None
        Country = None
        print('Warning: Address not updated')
    
    SQL = '''UPDATE Customers
             SET Address=coalesce(?,Address),
                 City=coalesce(?, City),
                 Country=coalesce(?, Country),
                 Phone=coalesce(?, Phone)
            WHERE CustID = ?'''
    
    cursor=conn.cursor()
    cursor.execute(SQL,(Address, City, Country, Phone, CustID))
    cust_id = cursor.lastrowid

In [43]:
UpdateCustomer(10, Address = 'fdfewtdf', City = None, Country = None, Phone = 'zzzzzz')

Phone does not contain only numerical values


In [45]:
UpdateCustomer(9, Address = 'fdfewtdf', City = 'aba', Country = 'oooo', Phone = 'zzzzzz')

Phone does not contain only numerical values


In [46]:
q = """
SELECT *
FROM Customers"""
pd.read_sql(q, conn)

,CustID,FirstName,LastName,Address,City,Country,Phone
0,1,Johan,van Niekerk,123 Kirkegata,Kristiansand,Norway,0047 91234432
1,2,Rayne,Reid,123 Hollendergata,Oslo,Norway,0047 12345678
2,3,Tor,Schultz,Smith Strasse,Berlin,Germany,0049 12345678
3,4,Buck,Rodgers,None,None,None,None
4,5,Darth,Vader,None,None,None,None
5,6,Hello,Friend,None,None,None,None
6,7,Hello,Friend,aabab,ababa,askjnfkjdnf,None
7,8,Hello,Friend,None,None,None,None
8,9,John,Smith,fdfewtdf,aba,oooo,8756666666
9,10,Hello,Friend,aabab,ababa,askjnfkjdnf,adfkjdhb
